# Implied Volatility Analysis
---

Full Name: In Pyoung Song <br> Course Code: ECO461H5 <br> Professor: Otto Yung <br> Due Date: 
## Summary 

We explore models that measure daily implied volatility and VaR using past data on the closing prices of the following indexes: GSPC, IXIC, DJI, XIU, XBB, COW, and XRE. The data looks between 1999-12-31 to 2020-12-31 daily closing prices for GSPC, IXIC, and DJI. As for the others, we observe the daily closing prices from 2011-01-04 to 2020-12-31. This project was to be done on Excel with. Answers to the assignment were provided via excel however, we explore additional models and topics such as copulas and correlated portfolios. The excel file can be found here (insert the link later). For the sake of extending the assignment's breadth, we also used the time series data for (insert which indices) and show additional results.

---

# Question 1 (25 Marks).
Estimate parameters using: (1) the maximum likelihood method and (2) the sum of the squared error
for the EWMA (i.e. λ and 1-λ) model on the S&P 500, NASDAQ and Dow Jones indices from January 3,
2000 to December 31, 2020. Briefly comment on the estimated parameters from both methods. Plot
the volatility and provide any comments (e.g. possible rationale for the volatility levels). (You can
create your own model or use the existing "template example" from class...) (Note: If necessary, insert a
constraint that λ is ≤ 0.98)

---

### i) We begin by importing the data and packages: 

In [1]:
# TODO: Importing the required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize

In [2]:
# TODO : Import and Observe The Data
q1_data = pd.read_csv("Question1.csv", float_precision = 'round_trip')
q1_data.head(10)

,Date,GSPC,IXIC,DJI
0,1999-12-31,1469.250000,4069.310059,11497.120117
1,2000-01-03,1455.219971,4131.149902,11357.509766
2,2000-01-04,1399.420044,3901.689941,10997.929688
3,2000-01-05,1402.109985,3877.540039,11122.650391
4,2000-01-06,1403.449951,3727.129883,11253.259766
5,2000-01-07,1441.469971,3882.620117,11522.559570
6,2000-01-10,1457.599976,4049.669922,11572.200195
7,2000-01-11,1438.560059,3921.189941,11511.080078
8,2000-01-12,1432.250000,3850.020020,11551.099609
9,2000-01-13,1449.680054,3957.209961,11582.429688


### ii) Estimate EMWA  and Tabulate Required Calculations

Setting the parameters: 

Set $\gamma = 0.94$ to initialize the paramater. According to the [RiskMetrics](https://www.msci.com/documents/10199/5915b101-4206-4ba0-aee2-3449d5c7e95a), $\gamma$ tends to be around 0.94.


In [3]:
gamma_ = 0.94

We now create the columns for:
* Required returns
* Squared returns
* Daily estimated variance (EMWA)
* Forward looking 1-day VAR (20 Trading Days)
* Error per annum (Forward looking 1-day VAR - EWMA Daily VAR)
* EWMA volatility per annum
* 20 - Day Forward Looking 1-Day VAR per Annum

The daily estimated variance (EWMA) is calculated by the following:

$$ \sigma_{n}^2 = \gamma\cdot\sigma_{n-1}^2 + (1-\gamma)\cdot\mu_{n-1}^2$$

Where $\mu_{n-1}^2$ are the one-day squared returns in period $n-1$, $\sigma^2_{n}$ is the one-day variance estimation in period n, and $\gamma$ is the parameter. 

To calculate EWMA volatility per annum and 20-day forward looking 1-day VAR per annum, we just multiply $\sqrt{252}$, where 252 is the number of trading days, to the 1-day calculations. 

In [4]:
# TODO: Create Columns for 'index_name Returns' 
def calc_pchange(index_name):
    q1_data[str(index_name) + ' Returns'] = q1_data[index_name].pct_change()
    return q1_data[str(index_name) + ' Returns']
calc_pchange('GSPC'), calc_pchange('IXIC'), calc_pchange('DJI')


# TODO: Create Columns for 'index_name Returns Squared'
def calc_rsq(index_name):
    q1_data[str(index_name) + ' Returns Squared'] = q1_data[ str(index_name) + " Returns"]**2
    return q1_data[str(index_name) + ' Returns Squared']
calc_rsq('GSPC'), calc_rsq('IXIC'), calc_rsq('DJI')


# TODO: Create Columns for 'index_name Daily VAR (EWMA)'
def calc_var (index_name, gamma):
    q1_data.loc[0, str(index_name) + ' Daily VAR (EWMA)'] = np.nan
    q1_data.loc[1, str(index_name) + ' Daily VAR (EWMA)'] = q1_data.loc[1 , str(index_name) + ' Returns Squared']
    
    for i in range(2, len(q1_data)):
        q1_data.loc[i, str(index_name) + ' Daily VAR (EWMA)'] = (
            gamma * q1_data.loc[i-1, str(index_name) + ' Daily VAR (EWMA)'] + 
            (1 - gamma) * q1_data.loc[i, str(index_name) + ' Returns Squared']
            )
    return q1_data[str(index_name) + ' Daily VAR (EWMA)']
calc_var('GSPC', gamma_), calc_var('IXIC', gamma_), calc_var('DJI', gamma_)


# TODO: Create a 20 trading days forward looking VAR
def for_var(index_name, forward_days):
    q1_data[str(index_name) + ' 20-Day Forward VAR'] = (q1_data[str(index_name) + ' Returns'].rolling(forward_days).var().shift(-forward_days))
    return q1_data[str(index_name) + ' 20-Day Forward VAR']
for_var('GSPC', 20), for_var('IXIC', 20), for_var('DJI', 20)


#TODO: Create Error column which is Forward - EWMA estimate
def error(index_name):
    q1_data[str(index_name) + ' Error'] = q1_data[str(index_name) + ' 20-Day Forward VAR'] - q1_data[str(index_name) + ' Daily VAR (EWMA)']
    return q1_data[str(index_name) + ' Error']
error('GSPC'), error('IXIC'), error('DJI')


#TODO: Create 1-Day EMWA Volatility per Annum
def annum_vol(index_name, trading_days):
    q1_data[str(index_name)+ ' EWMA Volatility'] = (trading_days * q1_data[str(index_name) + ' Daily VAR (EWMA)'])**(1/2)
    return q1_data[str(index_name)+ ' EWMA Volatility']
annum_vol('GSPC', 252), annum_vol('IXIC', 252), annum_vol('DJI', 252)


#TODO: Create 20-day forward looking volatility 
def for_vol(index_name):
    q1_data[str(index_name) + ' 20-Day Foward Volatility'] = q1_data[str(index_name) + ' 20-Day Forward VAR']**(1/2)
    return q1_data[str(index_name) + ' 20-Day Foward Volatility']
for_vol('GSPC'),for_vol('IXIC'),for_vol('DJI'),

 
# Print Columns: 
q1_data.iloc[:, [4,5,6,7,8,9,10,11,12,13,14,15]].head(5) 

,GSPC Returns,IXIC Returns,DJI Returns,GSPC Returns Squared,IXIC Returns Squared,DJI Returns Squared,GSPC Daily VAR (EWMA),IXIC Daily VAR (EWMA),DJI Daily VAR (EWMA),GSPC 20-Day Forward VAR,IXIC 20-Day Forward VAR,DJI 20-Day Forward VAR
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000268,0.000809,0.000213
1,-0.009549,0.015197,-0.012143,9.118549e-05,0.000231,0.000147,0.000091,0.000231,0.000147,0.000273,0.000840,0.000214
2,-0.038345,-0.055544,-0.031660,1.470314e-03,0.003085,0.001002,0.000174,0.000402,0.000199,0.000198,0.000673,0.000164
3,0.001922,-0.006190,0.011340,3.694786e-06,0.000038,0.000129,0.000164,0.000380,0.000195,0.000203,0.000716,0.000157
4,0.000956,-0.038790,0.011743,9.133209e-07,0.001505,0.000138,0.000154,0.000448,0.000191,0.000203,0.000612,0.000149


In [5]:
q1_data.iloc[:, [16,17,18,19,20,21,22,23,24]].head(5) 

,GSPC Error,IXIC Error,DJI Error,GSPC EWMA Volatility,IXIC EWMA Volatility,DJI EWMA Volatility,GSPC 20-Day Foward Volatility,IXIC 20-Day Foward Volatility,DJI 20-Day Foward Volatility
0,NaN,NaN,NaN,NaN,NaN,NaN,0.016357,0.028435,0.014605
1,0.000182,0.000609,0.000067,0.151587,0.241239,0.192765,0.016519,0.028980,0.014633
2,0.000024,0.000271,-0.000035,0.209359,0.318358,0.223796,0.014056,0.025940,0.012798
3,0.000040,0.000335,-0.000038,0.203119,0.309596,0.221414,0.014261,0.026755,0.012526
4,0.000050,0.000164,-0.000042,0.196966,0.335931,0.219471,0.014264,0.024743,0.012219


## iii) Calculating SSE When $\gamma = 0.94$

In [6]:
#TODO: Create Sum SQ Error Columns
def calc_sse(index_name):
    return (q1_data[str(index_name) + ' Error'] ** 2).sum()

#Print all SSE
print("""Sum of Squared Errors:
----------------------
GSPC {} 

IXIC: {}

DJI: {}
""".format(round(calc_sse('GSPC'),8), round(calc_sse('IXIC'),8), round(calc_sse('DJI'), 8)))

Sum of Squared Errors:
----------------------
GSPC 0.00043166 

IXIC: 0.00059702

DJI: 0.00047612



### iv) Minimizing MSE and Finding Optimal $\gamma$

To accomplish this, we will imitate the solver function in Excel using the Scipy.Optimize module. In order to do so, we need to define the objective function that we are minimizing. The objective function for each stock is: 

$$ MSE  = \frac{1}{n}\sum(\sigma_{20-Day Forward}^2 - (\lambda\cdot\sigma_{n-1}^2 + (1-\lambda)\cdot\mu_{n-1}^2) )^2 $$

Although it is much more convenient to use solver in Excel, we will demonstrate one way to go about solving for the parameter $\gamma$ to minimize $MSE$.

In [60]:
# TODO: Vectorize the exogenous variables + parameters
for_var = q1_data["GSPC 20-Day Foward Volatility"].to_numpy()
ewma_var = q1_data["GSPC Daily VAR (EWMA)"].shift(-1).to_numpy()
ret_sq = q1_data["GSPC Returns Squared"].to_numpy()
gamma = np.array([0.94])

def objective_function(gamma, for_var, ewma_var, ret_sq):
    sig_1 = for_var
    sig_2 = ewma_var
    sig_3 = ret_sq
    ga = gamma
    return (1/for_var.size) * np.nansum((sig_1 - (ga * sig_2 + (1-ga) * sig_3))**2)  

def constraint_1(gamma):
    ga = gamma
    return ga + (1-ga) - 1

bounds_gamma = (0.00000001, 0.99999999)

bounds = [bounds_gamma]

constraint_1 = {'type': 'eq', 'fun': constraint_1}

constraint = [constraint_1]

x0 = [0.94]

result = minimize(objective_function(np.empty(1), for_var, ewma_var, ret_sq), x0, method = 'SLSQP', bounds = bounds, constraints = constraint)

print(result)

TypeError: 'numpy.float64' object is not callable

## v) Visualization: Plotting Volatility

Here, we look to describe the data using a plot that shows the volatility over time. We find that each index converges in volatility levels with time. 

In [ ]:
# Plot Volatility
q1_data[['GSPC: EWMA Volatility', 'IXIC: EWMA Volatility', 'DJI: EWMA Volatility']].plot(figsize=(12, 8))

---



# Question 2: Estimating VaR

Using daily prices from January 2011to December 2018:

1. Calculate  daily  returns  then compute the  average  daily  return  and  standard  deviation  (i.e. volatility) foreach ETF

2. Calculate  the 1-Day  VaR  at  the  97.5%  confidence  level  for  each ETF

3. Construct  a  portfolio  of  these  four securities  with long-only  positions (i.e.  weight  greater  than 0%). 

You will assign the respective weights to each ETF in which the sum of all weights must add up to 1(note: No need to find optimal weights, just choose your own weights). You will then:

1. Compute the daily portfolio returns

2. Calculate the average daily return and standard deviation for the portfolio

3. Calculate the 1-Day VaR at the 97.5% confidence level for the portfolio(assume average daily return is zero)

4. Calculate  the  1-Day  Expected  Shortfall  at  the  97.5%  confidence  level  for  the  portfolio using the 97.5% 1-Day VaR that you just calculated

###  i) Import The Data (January 2011 - December 2018)

In [ ]:
# TODO: Import Question 2
q2 = pd.read_csv("Question2.csv", float_precision = 'round_trip').iloc[:2007]
# TODO: Filter data for data from 2011-01-04 - 2018-12-31
#print(q2)
q2

### ii) Table Calculations
<br>
We calculate the: 

* Daily returns
* Average returns
* Standard deviation  

For each ETF. 

In [ ]:
# TODO: Calculate daily returns


def daily_returns (stock): 
    return stock.pct_change()

xiu = daily_returns(q2["XIU"])
xbb = daily_returns(q2["XBB"])
cow = daily_returns(q2["COW"])
xre = daily_returns(q2["XRE"])

# TODO: Create dataframe for returns to use later
daily_returns = {'XIU':xiu, 'XBB':xbb, 'COW':cow, 'XRE':xre}
d_returns = pd.DataFrame(daily_returns, columns = ['XIU', 'XBB', 'COW', 'XRE'])

# TODO: Calculate average returns
def average_returns(stock):
    return stock.mean()

xiu_avg = average_returns(xiu)
xbb_avg = average_returns(xbb)
cow_avg = average_returns(cow)
xre_avg = average_returns(xre)

# TODO: Calculate standard deviation of returns
def stock_std (stock):
    return stock.std()

xiu_std = stock_std(xiu)
xbb_std = stock_std(xbb)
cow_std = stock_std(cow)
xre_std = stock_std(xre)

print("""

             XIU           XBB            COW            XRE 
-----------------------------------------------------------------
Avg Return:  {}        {}          {}        {}
-----------------------------------------------------------------
STD       :  {}        {}        {}        {}
""".format(round(xiu_avg,5), round(xbb_avg,5), round(cow_avg,5), round(xre_avg,5), round(xiu_std,5),
           round(xbb_std,5), round(cow_std,5), round(xre_std,5)))

print("""
Indices Returns
--------------------------------------------
{}""".format(d_returns.head(2504)))

### iii) Visualize Distribution of Returns

The distribution of returns of each ETF will identify which distribution would fit best for our VaR estimate. We will use the normal distribution for simplicity and note that the distributions are appear relatively normal. We do note however, based on the distributions shown below, they do not appear completely normal. Complex distributions and normalization will be dismissed. 

In [ ]:
d_returns.hist(figsize = (16,9), bins = 100, density = True)
plt.show()

### iv)  Calculate The 1-Day Value-at-Risk at a 97.5% Confidence Level
We will be using the following equation: 
<br>

### $$ VaR = \mu + \sigma \times Z$$

Where $\mu$ is the average return, $\sigma$ is the standard deviation of returns, and $Z$ is the Z score at the 97.5% confidence level. 

In [ ]:
# TODO: Calculate Z-Score
z_score = norm.ppf(0.975)

# TODO: Calculate VaR

def VaR (standard_deviation, average_return, confidence):
    return standard_deviation*norm.ppf(confidence)- average_return

xiu_VaR = VaR(xiu_std, xiu_avg, 0.975)
xbb_VaR = VaR(xbb_std, xbb_avg, 0.975)
cow_VaR = VaR(cow_std, cow_avg, 0.975)
xre_VaR = VaR(xre_std, xre_avg, 0.975)

print("""

Value at Risk (97.5%)
-------------------------------
XIU VaR: {}% 
XBB VaR: {}%
COW VaR: {}%
XRE VaR: {}%""".format(round(100*xiu_VaR,4), round(100*xbb_VaR,4), round(100*cow_VaR,4), round(100*xre_VaR,4)))

### v) Construct Portfolio: Table Calculations

Portfolio is constructed based on assigned weights. Create columns for:

* Portfolio Returns 
* Standard Deviation of Returns
* Average Return

We note that we make several assumptions in our calculations:
1. No asset correlation.
2. Only long-positions. 
3. Restricted from changing the weights of the portfolio at any time


In [ ]:
# TODO: Assign portfolio weights
weights = [0.25,0.25,0.25,0.25]
returns = d_returns

# TODO: Function that returns portfolio return. Assumes no asset correlation, long-position, no-reweighting. 
def portfolio_return (weights, returns):
    return (returns*weights).sum(axis = 1)

# TODO: Instantiate the portfolio_returns variable. 
pf_returns = pd.DataFrame(portfolio_return(weights, returns), columns = ["Portfolio Return"])
# TODO: Calculate average returns.
pf_mean = pf_returns.mean()
# TODO: Calculate portfolio SD. 
pf_sd = pf_returns.std()

# Print Portfolio Average Return & STD
print("""

Average Portfolio Return : {}

Portfolio STD            : {}""".format(pf_mean["Portfolio Return"], pf_sd["Portfolio Return"]))

In [ ]:
# Print Portfolio Return
print("""

Portfolio Returns
-----------------
{}""".format(pf_returns))

### vi) Calculate Portfolio 1-Day Value-at-Risk at a 97.5% Confidence Level

We will use the same equation we used earlier. To reiterate: 

$$ VaR = \mu + \sigma \times + Z$$

Where $\mu$ is the average return, $\sigma$ is the standard deviation of returns, and $Z$ is the Z score at the 97.5% confidence level. 


In [ ]:
# TODO: Calculate VaR
def pf_VaR(standard_deviation, average_return, confidence): 
        return -average_return + standard_deviation * norm.ppf(confidence)

pf_VaR = pf_VaR(pf_sd, pf_mean, 0.975)

print("""
      
The portfolio's VaR at a 97.5% confidence level is """ + str(pf_VaR["Portfolio Return"]) + ".") 

### vii) Calculate Portfolio 1-Day Expected Shortfall at a 97.5% Confidence Level

In order to calculate this, we will be making several assumptions: 

1. The 1-Day VaR is constant everyday and equalled to the overall average VaR. 
2. We calculate ES using the average of the returns that exceed VaR. 

We will calculate:

$$ ES = E(Loss | Loss \geq VaR)$$

In [ ]:
loss_exceed = []
for i in pf_returns["Portfolio Return"]:
    if i <= -pf_VaR["Portfolio Return"]: 
        loss_exceed.append(i)

exceptions = len(loss_exceed)
es = sum(loss_exceed)/len(loss_exceed)
print("""

# of Observed Exceptions   :  {}
Expected Shortfall (97.5%) : {}""".format(exceptions,es))


Therefore, we find that the total number of exceptions is __60__ and the expected shortfall at a 97.5% confidence level is __1.33%__ and VaR at the same confidence level is __0.009424%__. 

### i) Import Data (Jan 2019 - Dec 2020)

In [ ]:
# TODO: Import Question 2
q2 = pd.read_csv("Question2.csv", float_precision = 'round_trip').iloc[2006:]
# Show table
q2

### ii) Calculate Portfolio Returns

In [ ]:
# TODO: Calculate daily returns
def daily_returns_bt (stock): 
    return stock.pct_change()

xiu_bt = daily_returns_bt(q2["XIU"])
xbb_bt = daily_returns_bt(q2["XBB"])
cow_bt = daily_returns_bt(q2["COW"])
xre_bt = daily_returns_bt(q2["XRE"])

daily_returns_bt = {'XIU':xiu_bt, 'XBB':xbb_bt, 'COW':cow_bt, 'XRE':xre_bt}
d_returns_bt = pd.DataFrame(daily_returns_bt, columns = ['XIU', 'XBB', 'COW', 'XRE'])

# TODO: Assign portfolio weights
weights = [0.25,0.25,0.25,0.25]
returns = d_returns_bt

# TODO: Function that returns portfolio return. Assumes no asset correlation, long-position, no-reweighting. 
def portfolio_return_bt (weights, returns):
    return (returns*weights).sum(axis = 1)

# TODO: Instantiate the portfolio_returns variable. 
pf_returns_bt = pd.DataFrame(portfolio_return_bt(weights, returns), columns = ["Portfolio Return"])

### iii) Test for exceptions (2.5% chance of success)

In [ ]:
loss_exceed_bt = []
for i in pf_returns_bt["Portfolio Return"]:
    if i <= -pf_VaR["Portfolio Return"]: 
        loss_exceed.append(i)

exceptions = len(loss_exceed)
es = sum(loss_exceed)/len(loss_exceed)
print("""

# of Observed Exceptions   :  {}
Expected Shortfall (97.5%) : {}""".format(exceptions,es))